In [1]:
import os
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
import requests
from langchain_community.tools import DuckDuckGoSearchRun

load_dotenv()


True

In [2]:
# from langchain_huggingface import ChatHuggingFace

hugging_face_llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.1",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)

llm = ChatHuggingFace(llm=hugging_face_llm)

In [ ]:
# pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [5]:


search_tool = DuckDuckGoSearchRun()

In [6]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'

  response = requests.get(url)

  return response.json()

In [8]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [9]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


/opt/anaconda3/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [11]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [14]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Ireland, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
First, the user asked: "Find the capital of Ireland, then find its current weather condition." I need to break this down into steps.

Step 1: Find the capital of Ireland. I know from general knowledge that the capital of Ireland is Dublin. But since I have tools available, I should use them to confirm or if I didn't know, but I think it's fine to use my knowledge here. However, the instructions say to use the tools, so I might need to search for it to be thorough.

The tools I have are duckduckgo_search for current events and get_weather_data for weather. Since the capital is a fact, I could use duckduckgo_search to find it, but it might not be necessary if I already know. But to follow the format, I should use the tool.

The format requires me to think and take actions. So, I should start with the first part: finding the capital.

Thought: I need to find the capital of Ireland. I can use duckduckgo_search for this.

Action: duckduckgo_search

Ac

In [15]:
response['output']

'The capital of Ireland is Dublin. The current weather in Dublin is 15°C, overcast, with 88% humidity and wind speeds of 11 km/h.'